In [1]:
import win32com.client
import time
import shutil
import os
import codecs
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pywinauto import application
from pywinauto import timings

In [2]:
# 0: 날짜(ulong)
# 1:시간(long) - hhmm
# 2:시가(long or float)
# 3:고가(long or float)
# 4:저가(long or float)
# 5:종가(long or float)
# 6:전일대비(long or float) - 주) 대비부호(37)과 반드시 같이 요청해야 함
# 8:거래량(ulong or ulonglong) 주) 정밀도 만원 단위
# 9:거래대금(ulonglong)
# 10:누적체결매도수량(ulong or ulonglong) - 호가비교방식 누적체결매도수량
# 11:누적체결매수수량(ulong or ulonglong) - 호가비교방식 누적체결매수수량
#  (주) 10, 11 필드는 분,틱 요청일 때만 제공
# 12:상장주식수(ulonglong)
# 13:시가총액(ulonglong)
# 14:외국인주문한도수량(ulong)
# 15:외국인주문가능수량(ulong)
# 16:외국인현보유수량(ulong)
# 17:외국인현보유비율(float)
# 18:수정주가일자(ulong) - YYYYMMDD
# 19:수정주가비율(float)
# 20:기관순매수(long)
# 21:기관누적순매수(long)
# 22:등락주선(long)
# 23:등락비율(float)
# 24:예탁금(ulonglong)
# 25:주식회전율(float)
# 26:거래성립률(float)
# 37:대비부호(char) - 수신값은 GetHeaderValue 8 대비부호와 동일

In [3]:
# In[ ]:


class CREON(object):
    """대신증권 크레온 API"""
    
    def __init__(self):
        # 연결 여부 체크
        self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = self.objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            try:
                os.system("/Users/juhy9/Documents/GitHub/System/Quant/src/sudo/sudo_creon_plus_login.bat")
                time.sleep(100)
            except:
                exit()
     
    def setMethod(self, code, char, from_yyyymmdd=None, to_yyyymmdd=None, count=None):
        """
        count는 보통 상식의 데이터 개수가 아니다.
        여기서는 한번 요청 시 가져와지는 데이터의 개수이다.
        한번 요청 시 최대 2856개 가능하다.
        
        원하는 데이터 개수가 있으면 to_yyyymmdd 로 가져온 다음에 잘라서 사용한다.
        하루에 분단위 데이터가 381개이다. (* 마지막 10분은 동시호가)
        
        """
        # object 구하기
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.objStockChart.SetInputValue(0, code)  # 종목코드
        
        if to_yyyymmdd:
            self.objStockChart.SetInputValue(1, ord('1'))  # 요청 구분 '1': 기간, '2': 개수
            self.objStockChart.SetInputValue(2, from_yyyymmdd)  # To 날짜
            self.objStockChart.SetInputValue(3, to_yyyymmdd)  # From 날짜
        elif count:
            self.objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
            self.objStockChart.SetInputValue(4, count)  # 조회 개수
        else: raise print("기간을 입력해주세요.")
        
        if char == "m":
            # 날짜, 시간,시가,고가,저가,종가,거래량
            self.colnames = "날짜, 시간, 시가, 고가, 저가, 종가, 거래량".split(", ")
            self.objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])
        else:
            # 날짜,시가,고가,저가,종가,거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수
            self.colnames = "날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수".split(", ")
            self.objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9, 12, 13, 16, 20])
            
        self.objStockChart.SetInputValue(6, ord(char))  # '차트 주기 - 분/틱
        self.objStockChart.SetInputValue(7, 1)  # 분틱차트 주기
        
        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        
        
        
        self.data = {i: [] for i in self.colnames}
        
    def checkRequest(self):
        
        self.objStockChart.BlockRequest()
        
        rqStatus = self.objStockChart.GetDibStatus()
        
        if rqStatus != 0: 
            
            return False
        
#         else:
#             print("통신상태 양호, 누적 개수 {}".format(len(self.data["date"])))
        
        self.count = self.objStockChart.GetHeaderValue(3)
        
        if self.count <= 1: 
            
            return False
        
        return int(self.count)
    
    def checkRemainTime(self):
        
        # 연속 요청 가능 여부 체크
        remainTime = self.objCpCybos.LimitRequestRemainTime / 1000.
        remainCount = self.objCpCybos.GetLimitRemainCount(1)  # 시세 제한
        
        if remainCount <= 0:
            print("15초당 60건으로 제한합니다.")
            time.sleep(remainTime)
            
    
    def getStockPriceMin(self):
        
        while 1:
        
            self.checkRemainTime()
            rows = self.checkRequest()

            if rows:

                for i in range(rows):
                    
                    for idx, col in enumerate(self.colnames):
                    
                        self.data[col].append(self.objStockChart.GetDataValue(idx, i))
            else:

                break
                
    
        return self.data

In [4]:
# In[ ]:

SELECT_LIST = pd.read_csv("/Users/juhy9/Documents/GitHub/Quant/data/SELECT_LIST.csv")
SELECT_LIST = SELECT_LIST.values.tolist()

In [5]:
SELECT_LIST

[['SK하이닉스', 'A000660', 728002365, 60132995349000.0],
 ['삼성바이오로직스', 'A207940', 66165000, 38905020000000.0],
 ['삼성전자우', 'A005935', 822886700, 35178406425000.0],
 ['NAVER', 'A035420', 164263395, 32442020512500.0],
 ['셀트리온', 'A068270', 134706932, 28625223050000.0],
 ['LG화학', 'A051910', 70592343, 24954393250500.0],
 ['LG생활건강', 'A051900', 15618197, 21787384815000.0],
 ['현대차', 'A005380', 213668187, 19572005929200.0],
 ['삼성SDI', 'A006400', 68764530, 19185303870000.0],
 ['삼성물산', 'A028260', 189690043, 18912097287100.0],
 ['SK텔레콤', 'A017670', 80745711, 17198836443000.0],
 ['카카오', 'A035720', 86957175, 16000120200000.0],
 ['현대모비스', 'A012330', 95054694, 15921661245000.0],
 ['한국전력', 'A015760', 641964077, 15310843236450.0],
 ['POSCO', 'A005490', 87186835, 15170509290000.0],
 ['신한지주', 'A055550', 482432493, 14472974790000.0],
 ['KB금융', 'A105560', 415807920, 14220630864000.0],
 ['엔씨소프트', 'A036570', 21954022, 13831033860000.0],
 ['셀트리온헬스케어', 'A091990', 151062939, 13051837929600.0],
 ['SK', 'A034730', 7036

In [6]:
creon = CREON()

In [7]:
for yyyy in "2017 2018 2019".split(" "):
    
    for mm in "1 2 3 4 5 6 7 8 9 10 11 12".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for name, code, no, cap in tqdm(SELECT_LIST):

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):
                os.makedirs(savedir)

            if os.path.isfile(savefile):
                
                continue
                # os.remove(savefile)
                #read_table = pd.read_table(savefile, delimiter=" ")
                read_table = pd.DataFrame([])
                
            else:
                read_table = pd.DataFrame([])

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()
            
            DataFrame = pd.DataFrame(getStockPrice)
            DataFrame = DataFrame.iloc[::-1].reset_index(drop=True)

            if len(read_table) > 0:
                read_table = read_table.loc[~read_table.날짜.isin(DataFrame.날짜)].reset_index(drop=True)

            concat = pd.concat([read_table, DataFrame], 0).reset_index(drop=True)
            concat.to_csv(savefile, sep=" ", index=None)


2017 01


 50%|██████████████████                  | 211/420 [00:00<00:01, 174.38it/s]

15초당 60건으로 제한합니다.


 75%|███████████████████████████▊         | 315/420 [00:15<00:03, 29.10it/s]

15초당 60건으로 제한합니다.


...

100%|█████████████████████████████████████| 420/420 [03:04<00:00,  2.27it/s]


2019 12


 80%|█████████████████████████████▌       | 336/420 [02:28<00:50,  1.67it/s]

15초당 60건으로 제한합니다.


100%|█████████████████████████████████████| 420/420 [02:57<00:00,  2.36it/s]


In [8]:
for yyyy in "2020".split(" "):
    
    for mm in "1 2 3 4".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for name, code, no, cap in tqdm(SELECT_LIST):

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):
                os.makedirs(savedir)

            if os.path.isfile(savefile):
                continue
                os.remove(savefile)
                #read_table = pd.read_table(savefile, delimiter=" ")
                read_table = pd.DataFrame([])
            else:
                read_table = pd.DataFrame([])

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()
            
            DataFrame = pd.DataFrame(getStockPrice)
            DataFrame = DataFrame.iloc[::-1].reset_index(drop=True)

            if len(read_table) > 0:
                read_table = read_table.loc[~read_table.날짜.isin(DataFrame.날짜)].reset_index(drop=True)

            concat = pd.concat([read_table, DataFrame], 0).reset_index(drop=True)
            concat.to_csv(savefile, sep=" ", index=None)


2020 01


  0%|                                               | 0/420 [00:00<?, ?it/s]

15초당 60건으로 제한합니다.


100%|█████████████████████████████████████| 420/420 [03:11<00:00,  2.19it/s]


2020 02


...

100%|█████████████████████████████████████| 420/420 [03:45<00:00,  1.86it/s]


2020 04


 98%|████████████████████████████████████ | 410/420 [02:43<00:03,  2.57it/s]

15초당 60건으로 제한합니다.


100%|█████████████████████████████████████| 420/420 [02:48<00:00,  2.49it/s]
